In [46]:
from helpers import *
import pandas as pd
df = get_full_table('./input/training_variants','./input/training_text')
df2 = get_full_table('./input/test_variants','./input/test_text')
new_df = pd.concat([df, df2], axis=0)
gene = list(new_df['Gene'].unique())
len(gene)

1507

In [43]:
def get_from_path(path):
    """given a request path and return the 
    response and content"""
    import httplib2 as http
    import json
    try:
        from urlparse import urlparse
    except ImportError:
        from urllib.parse import urlparse

    headers = {
     'Accept': 'application/json',
    }

    uri = 'http://rest.genenames.org'

    target = urlparse(uri+path)
    method = 'GET'
    body = ''

    h = http.Http()

    response, content = h.request(
     target.geturl(),
     method,
     body,
     headers)
    return response, content
  
def get_gene_json(gene):
    """given gene, return a dict with info about it"""
    import time
    
    path = '/fetch/symbol/'+gene
    response, content = get_from_path(path)    
    if response['status'] == '200':
        # assume that content is a json reply
        # parse content with the json module 
        data = json.loads(content)
        #print(type(data))
        #print('Symbol:' + data['response']['docs'][0]['symbol'])
        #print('Symbol:' + data['response']['docs'][0]['name'])
        if data['response']['numFound'] != 0:
            print(data['response']['numFound'])
            return data
        else:
            path = '/fetch/prev_symbol/'+gene
            time.sleep(3)
            response, content = get_from_path(path)
            if response['status'] == '200':
                data = json.loads(content)
                if data['response']['numFound'] != 0:
                    return data
            else:
                print('Error detected: ' + response['status'])
                
    else:
        print('Error detected: ' + response['status'])
    

In [52]:
import time
import json
fail_list=[]
json_dict={}
for i, g in enumerate(gene[1001:]):
    data = get_gene_json(g)
    if data == None:
        fail_list.append(g)
    else:
        json_dict[g] = data
    time.sleep(3)
    
    if i%100 == 0:
        with open('./input/result'+str(i)+'.json', 'w') as f:
            json.dump(json_dict, f)
            
with open('./input/result.json', 'w') as f:
            json.dump(json_dict, f)  

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [55]:
with open('./input/result/result1000.json','r') as f:
    d = json.load(f)
with open('./input/result2/result.json','r') as f:
    d2 = json.load(f)

In [57]:
new_d = dict(d)
new_d.update(d2)

In [60]:
with open('./input/gene_query_result.json', 'w') as f:
    json.dump(new_d, f)